This is the beginning of the notebook.

##ADM4142-A Fundamentals of Data science <br>
The goal of this notebook is to retrieve and stage the source datasets into the format used in the dimensional model for analysis.

This notebook generates the Economy_dimension of the weather/tourism/economy data frame.

In [1168]:
import pandas as pd


In [1169]:
# Define the URL for the dataset
dataset_url = 'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/24100043_monthlyProvincial_tourists.csv'

# Load the dataset
tourism_data = pd.read_csv(dataset_url)

# Remove apostrophes from all cells
tourism_data = tourism_data.replace({"\"": ""}, regex=True)

#tourism_data.head(10)

<ipython-input-1169-289bb3f05feb>:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  tourism_data = pd.read_csv(dataset_url)


In [1170]:
#tourism_data.head()
tourism_data.tail()

,REF_DATE,GEO,DGUID,Traveller characteristics,Seasonal adjustment,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
135555,2021-12,Nunavut,2016A000262,Canadian tourists returning from United States...,Unadjusted,Persons,249,units,0,v114064913,13.10.1,1.0,NaN,NaN,NaN,0
135556,2021-12,Nunavut,2016A000262,Canadian tourists returning from United States...,Unadjusted,Persons,249,units,0,v114064915,13.11.1,0.0,NaN,NaN,NaN,0
135557,2021-12,Nunavut,2016A000262,Canadian tourists returning from countries oth...,Unadjusted,Persons,249,units,0,v114064917,13.12.1,0.0,NaN,NaN,NaN,0
135558,2021-12,Nunavut,2016A000262,Canadian tourists returning from countries oth...,Seasonally adjusted,Persons,249,units,0,v114064918,13.12.2,0.0,NaN,NaN,NaN,0
135559,2021-12,Nunavut,2016A000262,Canadian tourists returning from countries oth...,Unadjusted,Persons,249,units,0,v114064919,13.18.1,0.0,NaN,NaN,NaN,0


Now that we've imported the dataset, we need to transform it into the format that includes only the original columns.

In [1171]:

# Section 4: Filter rows by date
start_date = '1990-01'
end_date = '2023-12'
tourism_data['REF_DATE'] = pd.to_datetime(tourism_data['REF_DATE'])
tourism_data = tourism_data[(tourism_data['REF_DATE'] >= start_date) & (tourism_data['REF_DATE'] <= end_date)]

# Section 5: Filter rows by GEO column
valid_geo = ['Canada', 'Alberta', 'British Columbia', 'Ontario', 'Quebec']
tourism_data = tourism_data[tourism_data['GEO'].isin(valid_geo)]
#tourism_data = tourism_data[tourism_data['Seasonal adjustment'] == 'Unadjusted']

tourism_data.head()

,REF_DATE,GEO,DGUID,Traveller characteristics,Seasonal adjustment,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
22032,1990-01-01,Canada,2016A000011124,Total non resident tourists,Unadjusted,Persons,249,units,0,v129508,1.1.1,514393.0,NaN,NaN,NaN,0
22033,1990-01-01,Canada,2016A000011124,Total non resident tourists,Seasonally adjusted,Persons,249,units,0,v129566,1.1.2,1293413.0,NaN,NaN,NaN,0
22034,1990-01-01,Canada,2016A000011124,United States tourists,Unadjusted,Persons,249,units,0,v129509,1.2.1,402265.0,NaN,NaN,NaN,0
22035,1990-01-01,Canada,2016A000011124,United States tourists,Seasonally adjusted,Persons,249,units,0,v129567,1.2.2,1024597.0,NaN,NaN,NaN,0
22036,1990-01-01,Canada,2016A000011124,United States tourists entering by automobile,Unadjusted,Persons,249,units,0,v129559,1.13.1,252207.0,NaN,NaN,NaN,0


In [1172]:
# Section 6: Prepare columns for keys (to be mapped later)
tourism_data['Date_key'] = ""
tourism_data['Weather_key'] = ""
tourism_data['Location_key'] = ""
tourism_data['Economy_key'] = ""

# Section 7: Adjust columns based on Traveller characteristics
tourism_data['Total non-resident tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Total non resident tourists', 'VALUE']
tourism_data['United states tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'United States tourists', 'VALUE']
tourism_data['Non-US foreign tourists'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Tourists from countries other than United States', 'VALUE']
tourism_data['Canadian tourists returning from U.S.'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Canadian tourists returning from United States', 'VALUE']
tourism_data['Canadian tourists returning from abroad'] = tourism_data.loc[tourism_data['Traveller characteristics'] == 'Canadian tourists returning from countries other than United States', 'VALUE']
tourism_data['Seasonally adjusted'] = tourism_data['Seasonal adjustment']



# Section 9: Merge entries based on matching GEO and REF_DATE values
tourism_data = tourism_data.groupby(['REF_DATE', 'GEO', 'Seasonal adjustment'], as_index=False).agg({
    'Total non-resident tourists': 'sum',
    'United states tourists': 'sum',
    'Non-US foreign tourists': 'sum',
    'Canadian tourists returning from U.S.': 'sum',
    'Canadian tourists returning from abroad': 'sum',
    #'Seasonally adjusted': 'first',  # Since it's unadjusted, all values should be the same
})

# Section 8: Select entries with "Seasonal adjustment" value "Unadjusted"
unadjusted_data = tourism_data[tourism_data['Seasonal adjustment'] == 'Unadjusted']

## to be used later
adjusted_data = tourism_data.copy()
adjusted_data = adjusted_data[tourism_data['Seasonal adjustment'] == 'Seasonally adjusted']
##

# make a copy to perform manipulations
merged_data = unadjusted_data

# Section 10: Calculate "Total non-resident tourists"
merged_data['Total non-resident tourists'] = merged_data['United states tourists'] + merged_data['Non-US foreign tourists']

# Section 10: Fill missing values for other attributes
merged_data['Seasonal adjustment'].fillna("", inplace=True)
merged_data['Date_key'] = ""
merged_data['Weather_key'] = ""
merged_data['Location_key'] = ""
merged_data['Economy_key'] = ""


<ipython-input-1172-84aaf95b823f>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['Total non-resident tourists'] = merged_data['United states tourists'] + merged_data['Non-US foreign tourists']
<ipython-input-1172-84aaf95b823f>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['Seasonal adjustment'].fillna("", inplace=True)
<ipython-input-1172-84aaf95b823f>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [1173]:
merged_data.head(20)

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Date_key,Weather_key,Location_key,Economy_key
1,1990-01-01,Alberta,Unadjusted,16651.0,13541.0,3110.0,67654.0,12027.0,,,,
3,1990-01-01,British Columbia,Unadjusted,114903.0,79016.0,35887.0,194289.0,39542.0,,,,
5,1990-01-01,Canada,Unadjusted,514393.0,402265.0,112128.0,1071082.0,363144.0,,,,
7,1990-01-01,Ontario,Unadjusted,268094.0,217078.0,51016.0,504067.0,202654.0,,,,
9,1990-01-01,Quebec,Unadjusted,90107.0,70682.0,19425.0,211454.0,102062.0,,,,
11,1990-02-01,Alberta,Unadjusted,16777.0,13401.0,3376.0,65046.0,10815.0,,,,
13,1990-02-01,British Columbia,Unadjusted,113818.0,84239.0,29579.0,162381.0,31586.0,,,,
15,1990-02-01,Canada,Unadjusted,553673.0,446044.0,107629.0,934795.0,292403.0,,,,
17,1990-02-01,Ontario,Unadjusted,282245.0,230141.0,52104.0,454047.0,156871.0,,,,
19,1990-02-01,Quebec,Unadjusted,113204.0,92954.0,20250.0,158803.0,84008.0,,,,


Now, add the date dimension

In [1174]:
# Section 12: Load date dimension data
date_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/date.csv"
date_dimension = pd.read_csv(date_dimension_url)
date_dimension['date_iso'] = pd.to_datetime(date_dimension['date_iso'])

# Section 13: Merge with date dimension based on REF_DATE and date_iso
merged_data = pd.merge(merged_data, date_dimension, left_on='REF_DATE', right_on='date_iso', how='left')

# Section 14: Fill Date_key column with mapped values
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 15: Drop unnecessary columns
merged_data.drop(columns=['Date_key_x', 'Date_key_y', 'date_iso', 'day_of_week'], inplace=True)

In [1175]:
# Count the number of missing values in the Date_key column
missing_date_keys = merged_data['Date_key'].isna().sum()
print("Number of missing values in the Date_key column:", missing_date_keys)


Number of missing values in the Date_key column: 0


Now, add the location dimension

In [1176]:
# Section 17: Load location dimension data
location_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/location.csv"
location_dimension = pd.read_csv(location_dimension_url)
#location_dimension = pd.merge(location_dimension, pd dimension)
# Merge with location dimension based on Date_key
location_dimension = pd.merge(location_dimension, date_dimension[['Date_key', 'year']], on='Date_key', how='left')
#location_dimension['Date_key']

# Section 24: Fill Economy_key column with mapped values
#location_dimension['Date_key'] = location_dimension['Date_key_y']

location_dimension.head()

,Location_key,Date_key,location,population,year
0,1,1,Canada,27691138.0,1990
1,2,366,Canada,28037420.0,1991
2,3,731,Canada,28371264.0,1992
3,4,1097,Canada,28684764.0,1993
4,5,1462,Canada,29000663.0,1994


In [1177]:
# Section 18: Merge with location dimension based on Date_key and GEO
merged_data = pd.merge(merged_data, location_dimension, left_on=['year', 'GEO'], right_on=['year', 'location'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Location_key'] = merged_data['Location_key_y']
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 20: Drop unnecessary columns
merged_data.drop(columns=['Location_key_x', 'Location_key_y', 'Date_key_x', 'Date_key_y', 'location', 'population'], inplace=True)

# Section 21: Save the merged data as "tourism_fact_table.csv"
#merged_data.to_csv("tourism_fact_table.csv", index=False)


In [1178]:
# Count the number of missing values in the Location_key column
missing_location_keys = merged_data['Location_key'].isna().sum()
print("Number of missing values in the Location_key column:", missing_location_keys)


Number of missing values in the Location_key column: 0


In [1179]:
# Check the 10 entries starting from the 200th entry
#print(merged_data.iloc[100:200])

merged_data.head()

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Weather_key,Economy_key,date_yearmonth,year,month,day,Location_key,Date_key
0,1990-01-01,Alberta,Unadjusted,16651.0,13541.0,3110.0,67654.0,12027.0,,,1990-01,1990,1,1,69,1
1,1990-01-01,British Columbia,Unadjusted,114903.0,79016.0,35887.0,194289.0,39542.0,,,1990-01,1990,1,1,35,1
2,1990-01-01,Canada,Unadjusted,514393.0,402265.0,112128.0,1071082.0,363144.0,,,1990-01,1990,1,1,1,1
3,1990-01-01,Ontario,Unadjusted,268094.0,217078.0,51016.0,504067.0,202654.0,,,1990-01,1990,1,1,103,1
4,1990-01-01,Quebec,Unadjusted,90107.0,70682.0,19425.0,211454.0,102062.0,,,1990-01,1990,1,1,137,1


Now, add the economy dimension

In [1180]:
# Section 22: Load economy dimension data
economy_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/economy.csv"
#economy_dimension = pd.read_csv(economy_dimension_url)
economy_dimension = pd.read_csv(economy_dimension_url, usecols=['Date_key', 'Location_key', 'Economy_key'])
# Merge with date dimension based on Date_key to get year
economy_dimension = pd.merge(economy_dimension, date_dimension[['Date_key', 'year']], on='Date_key', how='left')

In [1181]:
# Section 23: Merge with economy dimension based on Date_key and Location_key
temp_merged_data = pd.merge(merged_data, economy_dimension, on=['year', 'Location_key'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Economy_key'] = temp_merged_data['Economy_key_y']
#merged_data['Date_key'] = merged_data['Date_key_y']

# Section 24: Fill Economy_key column with mapped values
#merged_data['Economy_key'] = temp_merged_data['Economy_key_y']

# Section 25: Drop unnecessary columns
#merged_data.drop(columns=['Economy_key_x', 'Economy_key_y', 'Date_key_x', 'Date_key_y'], inplace=True)

# Section 26: Save the merged data as "tourism_fact_table.csv"
#merged_data.to_csv("tourism_fact_table.csv", index=False)


In [1182]:
merged_data.tail(10)

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Weather_key,Economy_key,date_yearmonth,year,month,day,Location_key,Date_key
1910,2021-11-01,Alberta,Unadjusted,19683.0,14808.0,4875.0,61457.0,36045.0,,395.0,2021-11,2021,11,1,100,11324
1911,2021-11-01,British Columbia,Unadjusted,95230.0,69748.0,25482.0,95608.0,51371.0,,421.0,2021-11,2021,11,1,66,11324
1912,2021-11-01,Canada,Unadjusted,422246.0,303209.0,119037.0,497381.0,366902.0,,447.0,2021-11,2021,11,1,32,11324
1913,2021-11-01,Ontario,Unadjusted,212545.0,154111.0,58434.0,250368.0,189856.0,,473.0,2021-11,2021,11,1,134,11324
1914,2021-11-01,Quebec,Unadjusted,82432.0,53625.0,28807.0,75571.0,88849.0,,499.0,2021-11,2021,11,1,168,11324
1915,2021-12-01,Alberta,Unadjusted,35304.0,24891.0,10413.0,68694.0,49062.0,,525.0,2021-12,2021,12,1,100,11324
1916,2021-12-01,British Columbia,Unadjusted,155482.0,106843.0,48639.0,121827.0,68865.0,,551.0,2021-12,2021,12,1,66,11324
1917,2021-12-01,Canada,Unadjusted,619956.0,411289.0,208667.0,600326.0,503444.0,,577.0,2021-12,2021,12,1,32,11324
1918,2021-12-01,Ontario,Unadjusted,289445.0,198835.0,90610.0,295939.0,255819.0,,603.0,2021-12,2021,12,1,134,11324
1919,2021-12-01,Quebec,Unadjusted,124746.0,67050.0,57696.0,90487.0,125014.0,,629.0,2021-12,2021,12,1,168,11324


In [1183]:
# Count the number of missing values in the Location_key column
missing_economy_keys = merged_data['Economy_key'].isna().sum()
print("Number of missing values in the Economy_key column:", missing_economy_keys)


Number of missing values in the Economy_key column: 420


Finally, add the keys from Weather dimension. There are multiple Weather_keys corresponding to different cities from the same province when merging using Location_key and Date_key. These are combined into a single cell.


In [1184]:
import json

# Step 1: Load the weather dimension data
weather_dimension_url = "https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/weather_final.csv"
weather_dimension = pd.read_csv(weather_dimension_url, usecols=['Date_key', 'Location_key', 'Weather_key'])

# Step 2: Preprocess the Weather Data
# Aggregate Weather_key for each combination of Location_key and Date_key into a list, then convert to a JSON string
weather_aggregated = weather_dimension.groupby(['Date_key', 'Location_key']).agg(
    Weather_keys=('Weather_key', lambda x: json.dumps(list(x)))  # Convert the list of Weather_keys to JSON string
).reset_index()

In [1185]:
weather_aggregated.head()

,Date_key,Location_key,Weather_keys
0,1,35,"[2857, 3265, 3673]"
1,1,69,"[1, 409, 817]"
2,1,103,"[1633, 2449]"
3,1,137,"[1225, 2041]"
4,32,35,"[2858, 3266, 3674]"


In [1186]:
# Step 3: Merge with the Main Data
merged_data_with_weather = pd.merge(merged_data, weather_aggregated, on=['Location_key', 'Date_key'], how='left')
merged_data['Weather_key'] = merged_data_with_weather['Weather_keys']

# If needed, replace NaN values in Weather_keys with a default value, e.g., an empty list as a JSON string
#merged_data['Weather_key'].fillna(json.dumps([]), inplace=True)


In [1187]:
merged_data.tail()

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Weather_key,Economy_key,date_yearmonth,year,month,day,Location_key,Date_key
1915,2021-12-01,Alberta,Unadjusted,35304.0,24891.0,10413.0,68694.0,49062.0,"[373, 781, 1189]",525.0,2021-12,2021,12,1,100,11324
1916,2021-12-01,British Columbia,Unadjusted,155482.0,106843.0,48639.0,121827.0,68865.0,"[3229, 3637, 4045]",551.0,2021-12,2021,12,1,66,11324
1917,2021-12-01,Canada,Unadjusted,619956.0,411289.0,208667.0,600326.0,503444.0,NaN,577.0,2021-12,2021,12,1,32,11324
1918,2021-12-01,Ontario,Unadjusted,289445.0,198835.0,90610.0,295939.0,255819.0,"[2005, 2821]",603.0,2021-12,2021,12,1,134,11324
1919,2021-12-01,Quebec,Unadjusted,124746.0,67050.0,57696.0,90487.0,125014.0,"[1597, 2413]",629.0,2021-12,2021,12,1,168,11324


In [1188]:
# Count the number of missing values in the Location_key column
missing_weather_keys = merged_data['Weather_key'].isna().sum()
print("Number of missing values in the Weather_key column:", missing_weather_keys)


Number of missing values in the Weather_key column: 384


Completed. Note that There are no associated weather keys for rows whose location is "Canada", since there is no associated weather data included for the entire country.

Now we need to perform the same procedure for the seasonally adjusted data, and concatenate the two sets into one final dataset.

In [1189]:
#merged_data_with_weather.head()
final_data_unadjusted = merged_data.copy()

In [1190]:
# make a copy to perform manipulations
merged_data = adjusted_data

# Section 10: Calculate "Total non-resident tourists"
merged_data['Total non-resident tourists'] = merged_data['United states tourists'] + merged_data['Non-US foreign tourists']

# Section 10: Fill missing values for other attributes
merged_data['Seasonal adjustment'].fillna("", inplace=True)
merged_data['Date_key'] = ""
merged_data['Weather_key'] = ""
merged_data['Location_key'] = ""
merged_data['Economy_key'] = ""

### Date dimension
# Section 12: Load date dimension data

# Section 13: Merge with date dimension based on REF_DATE and date_iso
merged_data = pd.merge(merged_data, date_dimension, left_on='REF_DATE', right_on='date_iso', how='left')

# Section 14: Fill Date_key column with mapped values
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 15: Drop unnecessary columns
merged_data.drop(columns=['Date_key_x', 'Date_key_y', 'date_iso', 'day_of_week'], inplace=True)

### Location
# Section 18: Merge with location dimension based on Date_key and GEO
merged_data = pd.merge(merged_data, location_dimension, left_on=['year', 'GEO'], right_on=['year', 'location'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Location_key'] = merged_data['Location_key_y']
merged_data['Date_key'] = merged_data['Date_key_y']

# Section 20: Drop unnecessary columns
merged_data.drop(columns=['Location_key_x', 'Location_key_y', 'Date_key_x', 'Date_key_y', 'location', 'population'], inplace=True)

### Economy dimension
merged_data['Location_key'] = merged_data['Location_key'].astype('int64')

# Section 23: Merge with economy dimension based on Date_key and Location_key
temp_merged_data_2 = pd.merge(merged_data, economy_dimension, on=['year', 'Location_key'], how='left')

# Section 19: Fill Location_key column with mapped values
merged_data['Economy_key'] = temp_merged_data_2['Economy_key_y']

### Weather dimension
# Step 3: Merge with the Main Data
merged_data_with_weather_2 = pd.merge(merged_data, weather_aggregated, on=['Location_key', 'Date_key'], how='left')
merged_data['Weather_key'] = merged_data_with_weather_2['Weather_keys']

# If needed, replace NaN values in Weather_keys with a default value, e.g., an empty list as a JSON string
#merged_data['Weather_key'].fillna(json.dumps([]), inplace=True)


In [1191]:
merged_data.head()

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Weather_key,Economy_key,date_yearmonth,year,month,day,Location_key,Date_key
0,1990-01-01,Alberta,Seasonally adjusted,44048.0,34669.0,9379.0,65007.0,9813.0,"[1, 409, 817]",NaN,1990-01,1990,1,1,69,1
1,1990-01-01,British Columbia,Seasonally adjusted,274100.0,203443.0,70657.0,264000.0,31400.0,"[2857, 3265, 3673]",NaN,1990-01,1990,1,1,35,1
2,1990-01-01,Canada,Seasonally adjusted,1293412.0,1024597.0,268815.0,1439475.0,260819.0,NaN,NaN,1990-01,1990,1,1,1,1
3,1990-01-01,Ontario,Seasonally adjusted,699105.0,567129.0,131976.0,655467.0,147401.0,"[1633, 2449]",NaN,1990-01,1990,1,1,103,1
4,1990-01-01,Quebec,Seasonally adjusted,181668.0,135195.0,46473.0,299569.0,64915.0,"[1225, 2041]",NaN,1990-01,1990,1,1,137,1


In [1192]:
final_data_adjusted = merged_data

combined_data = pd.concat([final_data_unadjusted, final_data_adjusted], ignore_index=True)

In [1193]:
combined_data.tail(10)

,REF_DATE,GEO,Seasonal adjustment,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Weather_key,Economy_key,date_yearmonth,year,month,day,Location_key,Date_key
3830,2021-11-01,Alberta,Seasonally adjusted,30118.0,21810.0,8308.0,55287.0,31485.0,"[373, 781, 1189]",395.0,2021-11,2021,11,1,100,11324
3831,2021-11-01,British Columbia,Seasonally adjusted,147177.0,107935.0,39242.0,100761.0,53181.0,"[3229, 3637, 4045]",421.0,2021-11,2021,11,1,66,11324
3832,2021-11-01,Canada,Seasonally adjusted,611681.0,434459.0,177222.0,523267.0,393753.0,NaN,447.0,2021-11,2021,11,1,32,11324
3833,2021-11-01,Ontario,Seasonally adjusted,289534.0,210535.0,78999.0,262256.0,199499.0,"[2005, 2821]",473.0,2021-11,2021,11,1,134,11324
3834,2021-11-01,Quebec,Seasonally adjusted,123079.0,74974.0,48105.0,90112.0,108575.0,"[1597, 2413]",499.0,2021-11,2021,11,1,168,11324
3835,2021-12-01,Alberta,Seasonally adjusted,36799.0,28134.0,8665.0,67662.0,34168.0,"[373, 781, 1189]",525.0,2021-12,2021,12,1,100,11324
3836,2021-12-01,British Columbia,Seasonally adjusted,160663.0,116869.0,43794.0,131446.0,61819.0,"[3229, 3637, 4045]",551.0,2021-12,2021,12,1,66,11324
3837,2021-12-01,Canada,Seasonally adjusted,671938.0,466718.0,205220.0,644078.0,437576.0,NaN,577.0,2021-12,2021,12,1,32,11324
3838,2021-12-01,Ontario,Seasonally adjusted,320773.0,229798.0,90975.0,316614.0,216250.0,"[2005, 2821]",603.0,2021-12,2021,12,1,134,11324
3839,2021-12-01,Quebec,Seasonally adjusted,130891.0,71034.0,59857.0,97421.0,121750.0,"[1597, 2413]",629.0,2021-12,2021,12,1,168,11324


Combine, sort, and organize attributes and rows

In [1194]:
import numpy as np

# Final selection and sorting of the columns
final_data = combined_data[['Date_key', 'Weather_key', 'Location_key', 'Economy_key',
                                          'Total non-resident tourists', 'United states tourists',
                                          'Non-US foreign tourists', 'Canadian tourists returning from U.S.',
                                          'Canadian tourists returning from abroad', 'Seasonal adjustment']].copy()

final_data['Seasonal adjustment'] = final_data['Seasonal adjustment'].map({"Unadjusted": False, "Seasonally adjusted": True})
#final_data.rename(columns={'Weather_key': 'Weather_keys'}, inplace=True)
final_data.rename(columns={'Seasonal adjustment': 'Seasonally adjusted'}, inplace=True)

# Sort by Date_key (and any other keys as secondary sorts if you wish)
final_data = final_data.sort_values(by=['Date_key', 'Location_key'])

# Convert 'Economy_key' to numeric, coercing errors, then to int64 where possible
#final_data['Weather_key'] = pd.to_numeric(final_data['Weather_key'], errors='ignore').astype('Int64')
final_data['Economy_key'] = pd.to_numeric(final_data['Economy_key'], errors='ignore').astype('Int64')
# Replace NaN values in "Weather_key" column with pd.NA
#final_data['Weather_key'] = final_data['Weather_key'].replace(np.nan, pd.NA)

# Assuming df is your DataFrame and it has a column named "Weather_key"
# First, ensure the column is of a type that supports pd.NA. If it's not, this step can be omitted.
final_data['Economy_key'] = final_data['Economy_key'].astype('object')

# Replace pd.NA with np.nan
final_data['Economy_key'] = final_data['Economy_key'].fillna(np.nan)


# Reset the index after sorting
final_data.reset_index(drop=True, inplace=True)

# Display the first few rows of the final sorted dataset
#print(final_data.head())

View final data

In [1195]:
final_data.head(20)

,Date_key,Weather_key,Location_key,Economy_key,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Seasonally adjusted
0,1,NaN,1,NaN,514393.0,402265.0,112128.0,1071082.0,363144.0,False
1,1,NaN,1,NaN,553673.0,446044.0,107629.0,934795.0,292403.0,False
2,1,NaN,1,NaN,691807.0,557339.0,134468.0,1479849.0,367128.0,False
3,1,NaN,1,NaN,856233.0,675367.0,180866.0,1463662.0,259138.0,False
4,1,NaN,1,NaN,1318376.0,1046317.0,272059.0,1291017.0,227538.0,False
5,1,NaN,1,NaN,1974020.0,1602321.0,371699.0,1201980.0,207527.0,False
6,1,NaN,1,NaN,2591846.0,2094914.0,496932.0,2283299.0,245025.0,False
7,1,NaN,1,NaN,2686411.0,2204680.0,481731.0,2425170.0,331231.0,False
8,1,NaN,1,NaN,1601550.0,1258964.0,342586.0,1539746.0,257522.0,False
9,1,NaN,1,NaN,1039257.0,831760.0,207497.0,1334601.0,218831.0,False


In [1196]:
final_data.tail(20)

,Date_key,Weather_key,Location_key,Economy_key,Total non-resident tourists,United states tourists,Non-US foreign tourists,Canadian tourists returning from U.S.,Canadian tourists returning from abroad,Seasonally adjusted
3820,11324,"[1597, 2413]",168,369.0,12062.0,5610.0,6452.0,17341.0,13815.0,False
3821,11324,"[1597, 2413]",168,499.0,14625.0,6638.0,7987.0,14942.0,16244.0,False
3822,11324,"[1597, 2413]",168,629.0,35483.0,18185.0,17298.0,29536.0,44508.0,False
3823,11324,"[1597, 2413]",168,109.0,79037.0,56342.0,22695.0,48423.0,116111.0,False
3824,11324,"[1597, 2413]",168,239.0,95677.0,54920.0,40757.0,36997.0,86753.0,False
3825,11324,"[1597, 2413]",168,369.0,103211.0,57413.0,45798.0,49491.0,76196.0,False
3826,11324,"[1597, 2413]",168,499.0,82432.0,53625.0,28807.0,75571.0,88849.0,False
3827,11324,"[1597, 2413]",168,629.0,124746.0,67050.0,57696.0,90487.0,125014.0,False
3828,11324,"[1597, 2413]",168,499.0,16493.0,7492.0,9001.0,17734.0,36994.0,True
3829,11324,"[1597, 2413]",168,629.0,8973.0,3694.0,5279.0,14408.0,18150.0,True


In [1197]:
# Save to file
final_data.to_csv("tourism_fact_table.csv", index=False)